In [25]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)
    
    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)
    
    
def merge_notes(notes_dict, max_offset):
    for offset in notes_dict:
        element_list = notes_dict[offset]
        if len(element_list) > 1:
            merged_notes = []
            for element in element_list:
                if isinstance(element, note.Note):
                    merged_notes.append(element)
                elif isinstance(element, chord.Chord):
                    for n in element.normalOrder:
                        merged_notes.append(note.Note(n))
            c = chord.Chord(merged_notes)
            notes_dict[offset] = c
        else:
            notes_dict[offset] = element_list[0]           
            
    ret = []
    for i in np.arange(0, max_offset, 0.5):
        if i in notes_dict:
            element = notes_dict[i]
            if isinstance(element, note.Note):
                ret.append(str(element.pitch))
            else:
                ret.append('.'.join(str(n) for n in element.normalOrder))    
        else:
            ret.append("---")
    return ret
    

def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
        notes_dict = {}
        max_offset = 0
        for element in notes_to_parse:        
            if isinstance(element, note.Note) or isinstance(element, chord.Chord):  
                if element.offset not in notes_dict:
                    notes_dict[element.offset] = []
                notes_dict[element.offset].append(element)
                max_offset = element.offset    
        ret = merge_notes(notes_dict, max_offset)
        #print(len(ret))
        notes += ret
        #print(len(notes))
    
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    #model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)


In [26]:
#print(get_notes())

In [27]:
train_network()

Parsing midi_songs/Ff7-Cinco.mid
Parsing midi_songs/waltz_de_choco.mid
Parsing midi_songs/Ff4-BattleLust.mid
Parsing midi_songs/dontbeafraid.mid
Parsing midi_songs/electric_de_chocobo.mid
Parsing midi_songs/costadsol.mid
Parsing midi_songs/Ff7-Jenova_Absolute.mid
Parsing midi_songs/tifap.mid
Parsing midi_songs/Still_Alive-1.mid
Parsing midi_songs/bcm.mid
Parsing midi_songs/ff7-mainmidi.mid
Parsing midi_songs/ultros.mid
Parsing midi_songs/0fithos.mid
Parsing midi_songs/mining.mid
Parsing midi_songs/Ff7-One_Winged.mid
Parsing midi_songs/FFIX_Piano.mid
Parsing midi_songs/balamb.mid
Parsing midi_songs/caitsith.mid
Parsing midi_songs/path_of_repentance.mid
Parsing midi_songs/ff1battp.mid
Parsing midi_songs/Finalfantasy6fanfarecomplete.mid
Parsing midi_songs/Oppressed.mid
Parsing midi_songs/relmstheme-piano.mid
Parsing midi_songs/ff4-fight1.mid
Parsing midi_songs/Finalfantasy5gilgameshp.mid
Parsing midi_songs/FF4.mid
Parsing midi_songs/Final_Fantasy_Matouyas_Cave_Piano.mid
Parsing midi_songs

46310/46310 [==============================] - 166s 4ms/step - loss: 1.3793 - acc: 0.7297
Epoch 49/200
46310/46310 [==============================] - 166s 4ms/step - loss: 1.3553 - acc: 0.7355
Epoch 50/200
46310/46310 [==============================] - 166s 4ms/step - loss: 1.3199 - acc: 0.7451
Epoch 51/200
46310/46310 [==============================] - 166s 4ms/step - loss: 1.2815 - acc: 0.7520
Epoch 52/200
46310/46310 [==============================] - 166s 4ms/step - loss: 1.2583 - acc: 0.7580
Epoch 53/200
46310/46310 [==============================] - 166s 4ms/step - loss: 1.2248 - acc: 0.7639
Epoch 54/200
46310/46310 [==============================] - 166s 4ms/step - loss: 1.2033 - acc: 0.7679
Epoch 55/200
46310/46310 [==============================] - 166s 4ms/step - loss: 1.1833 - acc: 0.7728
Epoch 56/200
46310/46310 [==============================] - 166s 4ms/step - loss: 1.1462 - acc: 0.7808
Epoch 57/200
46310/46310 [==============================] - 166s 4ms/step - loss: 1.13

46310/46310 [==============================] - 166s 4ms/step - loss: 0.7490 - acc: 0.8596
Epoch 128/200
46310/46310 [==============================] - 166s 4ms/step - loss: 0.7363 - acc: 0.8629
Epoch 129/200
46310/46310 [==============================] - 166s 4ms/step - loss: 0.7383 - acc: 0.8619
Epoch 130/200
46310/46310 [==============================] - 166s 4ms/step - loss: 0.7502 - acc: 0.8569
Epoch 131/200
46310/46310 [==============================] - 166s 4ms/step - loss: 0.7356 - acc: 0.8632
Epoch 132/200
46310/46310 [==============================] - 166s 4ms/step - loss: 0.7475 - acc: 0.8591
Epoch 133/200
46310/46310 [==============================] - 166s 4ms/step - loss: 0.7407 - acc: 0.8604
Epoch 134/200
46310/46310 [==============================] - 166s 4ms/step - loss: 0.7452 - acc: 0.8598
Epoch 135/200
46310/46310 [==============================] - 166s 4ms/step - loss: 0.7622 - acc: 0.8544
Epoch 136/200
46310/46310 [==============================] - 166s 4ms/step - l

In [5]:
""" This module generates notes for a midi file using the
    trained neural network """
import pickle
import numpy as np
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation

def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights('weights-improvement-109-0.9271-bigger.hdf5')

    return model

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        elif pattern == '---':
            pass
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.25

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')
    

In [11]:
generate()